In [6]:
from vnstock3 import Vnstock

### Quote

In [7]:
from vnstock3 import Vnstock
stock = Vnstock().stock(symbol='ACB', source='VCI')

2024-09-07 23:49:52,207 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [24]:
stock.quote.intraday(page_size=10000, last_time='2024-09-07 14:29:52')['match_type'].value_counts()

match_type
Buy        1003
Sell        840
ATO/ATC       2
Name: count, dtype: int64

### Trading

In [10]:
stock.listing.all_symbols()

,ticker,organ_name
0,A32,Công ty Cổ phần 32
1,AAA,Công ty Cổ phần Nhựa An Phát Xanh
2,AAM,Công ty Cổ Phần Thủy Sản MeKong
3,AAS,Công ty Cổ phần Chứng khoán SmartInvest
4,AAT,Công ty Cổ phần Tập đoàn Tiên Sơn Thanh Hóa
...,...,...
1593,XPH,Công ty Cổ phần Xà phòng Hà Nội
1594,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
1595,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
1596,YEG,Công ty Cổ phần Tập đoàn Yeah1


In [12]:
stock.listing.symbols_by_group('HOSE')

0      AAA
1      AAM
2      AAT
3      ABR
4      ABS
      ... 
411    VTB
412    VTO
413    VTP
414    YBM
415    YEG
Name: symbol, Length: 416, dtype: object

In [32]:
df = stock.quote.history(start='2024-06-01', end='2024-08-20', interval='15m')
df

,time,open,high,low,close,volume
0,2024-06-03 09:15:00,24.85,24.95,24.70,24.75,392800
1,2024-06-03 09:30:00,24.75,24.80,24.60,24.65,1016600
2,2024-06-03 09:45:00,24.65,24.70,24.60,24.60,544000
3,2024-06-03 10:00:00,24.60,24.75,24.60,24.70,543600
4,2024-06-03 10:15:00,24.70,24.75,24.65,24.70,497300
...,...,...,...,...,...,...
7506,2024-08-20 13:45:00,24.10,24.25,24.10,24.25,721700
7507,2024-08-20 14:00:00,24.25,24.30,24.15,24.15,496700
7508,2024-08-20 14:15:00,24.15,24.20,24.15,24.15,634500
7509,2024-08-20 14:30:00,NaN,NaN,NaN,NaN,0


In [17]:
# Bước giá và khối lượng giao dịch: realtime
stock.quote.price_depth('ACB')

,price,acc_volume,acc_buy_volume,acc_sell_volume,acc_undefined_volume
0,24300,78000,78000,0,0
1,24250,425700,155000,270700,0
2,24200,426900,340000,86900,0
3,24150,1032200,437200,595000,0
4,24100,1757700,952400,482600,322700
5,24050,1368500,577100,758800,32600
6,24000,324000,21300,302700,0


In [28]:
company = Vnstock().stock(symbol='ACB', source='TCBS').company

In [29]:
company.shareholders()

,share_holder,share_own_percent
0,Vietnam Enterprise Investments Limited,0.0548
1,Trần Hùng Huy,0.0394
2,Đặng Thu Thủy,0.0137
3,Đặng Thu Hà,0.0119
4,Nguyễn Thúy Lan,0.0096
5,Nguyễn Thùy Hương,0.0090
6,Trần Phú Mỹ,0.0083
7,Nguyễn Đức Kiên,0.0046
8,Đặng Phú Vinh,0.0042
9,Khác,0.2548
